In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
# from datetime import date, timedelta


StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 5, Finished, Available, Finished)

In [2]:
# Remove this before running Data Factory Pipeline
# start_date = date.today() - timedelta(7)

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 6, Finished, Available, Finished)

In [3]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 7, Finished, Available, Finished)

In [4]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example
    >>> get_country_code(48.777, 56.9448)
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 8, Finished, Available, Finished)

In [5]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 9, Finished, Available, Finished)

In [6]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 10, Finished, Available, Finished)

In [7]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class',
                                when(col("sig") < 100, "Low").\
                                when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                otherwise("High")
                                )

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 11, Finished, Available, Finished)

In [8]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, d3202b74-fb74-4a0f-beaa-28f21dbb2a3c, 12, Finished, Available, Finished)